<a href="https://colab.research.google.com/github/jetsonai/HK_LSTMSenfuClass/blob/main/Day1/CNN/%5B1%5DVGG_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGGNet for Color Images (RGB, CIFAR-10)
## TensorFlow/Keras version

In [41]:
import os, random, math
import numpy as np
import tensorflow as tf
from dataclasses import dataclass

## 1) Reproducibility & Device

In [42]:
def fix_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
fix_seed(42)


In [43]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # set memory growth instead of pre-allocating all VRAM
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU detected: {tf.config.experimental.get_device_details(gpus[0]).get('device_name','GPU')}")
    except Exception as e:
        print("⚠️ Could not set memory growth:", e)
else:
    print("⚠️ No GPU detected. Using CPU.")

✅ GPU detected: Tesla T4


## 2) Hyper-params & constants

In [44]:
@dataclass
class CFG:
    img_size: int = 32
    num_classes: int = 10
    width: int = 16           # base width (like PyTorch code)
    p_dropout: float = 0.25   # for BN/Dropout variant
    batch_size: int = 64
    epochs: int = 50
    lr: float = 1e-2
    use_bn_dropout: bool = False   # toggle VGG16withBN+Dropout variant
    out_dir: str = "ckpt_tf/vgg16_rgb_bn"  # folder will be renamed based on flag

In [45]:
CFG = CFG()
CFG.out_dir = f"ckpt_tf/{'vgg16_rgb_bn' if CFG.use_bn_dropout else 'vgg16_rgb'}"
os.makedirs(CFG.out_dir, exist_ok=True)

In [46]:
# CIFAR-10 mean/std in RGB order (same as PyTorch version)
CIFAR10_MEAN = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
CIFAR10_STD  = np.array([0.2470, 0.2435, 0.2616], dtype=np.float32)

## 3) Data: CIFAR-10 (RGB)

In [47]:
def _normalize(img):
    # img: float32 in [0,1], shape [H, W, 3]
    return (img - CIFAR10_MEAN) / CIFAR10_STD

def _augment(img, lbl):
    # simple augmentation similar to RandomHorizontalFlip + resize
    img = tf.image.resize(img, (CFG.img_size, CFG.img_size))
    img = tf.image.random_flip_left_right(img)
    img = _normalize(img)
    return img, lbl

def _preprocess(img, lbl):
    img = tf.image.resize(img, (CFG.img_size, CFG.img_size))
    img = _normalize(img)
    return img, lbl

In [48]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.squeeze().astype(np.int64)
y_test  = y_test.squeeze().astype(np.int64)

x_train = (x_train.astype(np.float32) / 255.0)
x_test  = (x_test.astype(np.float32) / 255.0)

In [49]:
train_ds = (tf.data.Dataset
            .from_tensor_slices((x_train, y_train))
            .shuffle(10_000, seed=42, reshuffle_each_iteration=True)
            .map(_augment, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(CFG.batch_size)
            .prefetch(tf.data.AUTOTUNE))

test_ds = (tf.data.Dataset
           .from_tensor_slices((x_test, y_test))
           .map(_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(CFG.batch_size)
           .prefetch(tf.data.AUTOTUNE))

## 4) VGG-style building blocks

In [50]:
from tensorflow.keras import layers, models, regularizers

def ConvBlock(x, out_ch, num_layers=2, use_bn=False):
    for _ in range(num_layers):
        x = layers.Conv2D(out_ch, kernel_size=3, strides=1, padding="same",
                          use_bias=not use_bn,
                          kernel_initializer="he_normal")(x)
        if use_bn:
            x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    return x

In [51]:
def LinearBlock(x, out_dim, p_drop=0.0, use_dropout=False):
    # PyTorch code uses 3 linear layers ending with num_classes
    # We already have GAP -> (width*4) features; mirror that:
    x = layers.Dense(x.shape[-1], kernel_initializer="he_normal")(x)
    if use_dropout:
        x = layers.Dropout(p_drop)(x)
    x = layers.ReLU()(x)

    x = layers.Dense(x.shape[-1], kernel_initializer="he_normal")(x)
    if use_dropout:
        x = layers.Dropout(p_drop)(x)
    x = layers.ReLU()(x)

    x = layers.Dense(out_dim)(x)
    return x

In [52]:
def build_vgg16(img_channels=3, width=16, num_classes=10, use_bn_dropout=False, p=0.25):
    inp = layers.Input(shape=(CFG.img_size, CFG.img_size, img_channels))

    # Five conv blocks with 2,2,3,3,3 conv layers (as in the PyTorch version)
    x = ConvBlock(inp, width,     num_layers=2, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*2,   num_layers=2, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    # GAP to mimic AdaptiveAvgPool2d(1).flatten
    x = layers.GlobalAveragePooling2D()(x)

    # Classifier (with optional Dropout, as in VGG16withBN)
    x = LinearBlock(x, num_classes, p_drop=p, use_dropout=use_bn_dropout)

    return models.Model(inp, x, name="VGG16_TF" + ("_BN" if use_bn_dropout else ""))

In [53]:
model = build_vgg16(img_channels=3,
                    width=CFG.width,
                    num_classes=CFG.num_classes,
                    use_bn_dropout=CFG.use_bn_dropout,
                    p=CFG.p_dropout)

## 5) Compile & Callbacks

In [54]:
# Using SGD(m=0.9) to match the PyTorch script
opt = tf.keras.optimizers.SGD(learning_rate=CFG.lr, momentum=0.9)

In [55]:
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])

ckpt_best = os.path.join(CFG.out_dir, "best.weights.h5")
ckpt_last = os.path.join(CFG.out_dir, "latest.weights.h5")

In [56]:
cbs = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_best, monitor="val_acc",
                                       save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(ckpt_last, monitor="val_acc",
                                       save_best_only=False, save_weights_only=True, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
]


In [57]:
model.summary()

Model: "VGG16_TF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_45 (ReLU)                 │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 32, 32, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_46 (ReLU)                 │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 16, 16, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_47 (ReLU)                 │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_48 (ReLU)                 │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_49 (ReLU)                 │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_50 (ReLU)                 │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_51 (ReLU)                 │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_52 (ReLU)                 │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_53 (ReLU)                 │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_54 (ReLU)                 │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 2, 2, 64)       │             

 Total params: 339,546 (1.30 MB)

 Trainable params: 339,546 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

## 6) Train

In [58]:
hist = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=CFG.epochs,
    callbacks=cbs
)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.2282 - loss: 2.0276
Epoch 1: val_acc improved from -inf to 0.43100, saving model to ckpt_tf/vgg16_rgb/best.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - acc: 0.2284 - loss: 2.0273 - val_acc: 0.4310 - val_loss: 1.5066 - learning_rate: 0.0100
Epoch 2/50
778/782 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.4831 - loss: 1.4103
Epoch 2: val_acc improved from 0.43100 to 0.56680, saving model to ckpt_tf/vgg16_rgb/best.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - acc: 0.4832 - loss: 1.4099 - val_acc: 0.5668 - val_loss: 1.1927 - learning_rate: 0.0100
Epoch 3/50
776/782 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.5687 - loss: 1.2020
Epoch 3: val_acc improved from 0.56680 to 0.60840, saving model to ckpt_tf/vgg16_rgb/best.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - acc: 0.5688 - loss: 1.2017 - val_acc: 0.6084 - val_loss: 1.1060 - learning_rate: 0.0100
Epoch 4/50
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0

In [59]:
# Save training curves
import matplotlib.pyplot as plt

plt.figure()
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="val")
plt.title("Loss (Training vs. Validation)")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(CFG.out_dir, "loss.png")); plt.close()

plt.figure()
plt.plot(hist.history["acc"], label="train")
plt.plot(hist.history["val_acc"], label="val")
plt.title("Accuracy (Training vs. Validation)")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(CFG.out_dir, "accuracy.png")); plt.close()

## 7) Inference helper (evaluate)

In [60]:
def inference(weights_path: str, batch_size: int = 64):
    """Load weights and evaluate on CIFAR-10 test set."""
    tmp_model = build_vgg16(img_channels=3, width=CFG.width,
                            num_classes=CFG.num_classes,
                            use_bn_dropout=CFG.use_bn_dropout, p=CFG.p_dropout)
    tmp_model.compile(optimizer="sgd",
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])
    tmp_model.load_weights(weights_path)
    print(f"Loaded weights from: {weights_path}")
    results = tmp_model.evaluate(test_ds, verbose=1)
    print(f"[Test] Loss: {results[0]:.4f} | Acc.: {results[1]:.4f}")
    return results

In [61]:
from google.colab import files

def save_trained_model(save_path):
  model.save_weights(save_path)
  print(f"✅ 모델 가중치 저장 완료: {save_path}")

  files.download(save_path)

#### # Example:
inference(ckpt_last)

inference(ckpt_best)
###

In [62]:
if __name__ == "__main__":
    # quick test: evaluate last checkpoint
    try:
        inference(ckpt_last)
        save_path = "/content/ckpt_tf/vgg16_rgb_final.weights.h5"
        save_trained_model(save_path)

    except Exception as e:
        print("Inference skipped (weights might not be saved yet):", e)

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loaded weights from: ckpt_tf/vgg16_rgb/latest.weights.h5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - acc: 0.8019 - loss: 0.7331
[Test] Loss: 0.7380 | Acc.: 0.7952
✅ 모델 가중치 저장 완료: /content/ckpt_tf/vgg16_rgb_final.weights.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>